In [1]:
from transformers import AutoTokenizer,AutoModelForTokenClassification
import torch
import re
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
df = pd.read_excel("data training")

In [ ]:
df['berita'] = df['berita'].replace({'"':' ',
                                 ':':' ',
                                 ';':' ',
                                 '#':' ',
                                 '@':' ',
                                 '_':' ',
                                 ',': ' ',
                                 "'": ' ',
                                  }, regex=True)

df['berita'] = df['berita'].str.replace(r'[https]+[?://]+[^\s<>"]+|www\.[^\s<>"]+[?()]+[(??)]+[)*]+[(\xa0]+[-&gt...]', "",regex=True)

df['berita'] = df['berita'].replace('\n','.', regex=True)

df['berita'] = df['berita'].replace({'(/)':'','\(':'','\)':''},regex=True)
df['berita'] = df['berita'].replace('[\:"]','',regex =True)

df['berita'] = df['berita'].replace('\s+', ' ', regex=True)
letters_only = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(df['berita']))
df['berita'] = df['berita'].str.strip()
df['berita'] = df['berita'].str.lower()